In [39]:
'''
File : Notebook Vision
Author : Benoît Gallois
Date : 19 nov 2023
Definition of the function that can detect the obstacles, the robot and the goal zone.
'''

import cv2
import numpy as np


IMAGE_PATH = "map2.jpeg"

# Definition of the colours thresholds
LOWER_RED = np.array([0, 100, 100])
UPPER_RED = np.array([10, 255, 255])

LOWER_BLUE = np.array([100, 100, 100])
UPPER_BLUE = np.array([140, 255, 255])

LOWER_GREEN = np.array([40, 40, 40])
UPPER_GREEN = np.array([80, 255, 255])

LOWER_BLACK = np.array([0, 0, 0])
UPPER_BLACK = np.array([179, 255, 30])

# Definition of the size of contours considered as noise
NOISY_CONTOUR_LENGHT = 2000



def detect_area(image_path, lower_colour, upper_colour):

    '''
    @brief   Detects areas corresponding to a color and returns the coordinates of the vertices of these areas.
 
    @param   image_path   -> "folder_of_image/name_image.jpeg"
             lower_colour -> LOWER_RED, LOWER_BLACK, LOWER_BLUE, LOWER_GREEN
             upper_colour -> UPPER_RED, UPPER_BLACK, UPPER_BLUE, UPPER_GREEN     
    '''
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape      # Give the size of the image

    # Converts the image in the HSV space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Filter the image to retain only pixels of the desired color
    mask = cv2.inRange(hsv, lower_colour, upper_colour)

    # Blur masks to reduce noise
    blurred_mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # Find contours in the filtered mask
    contours, _ = cv2.findContours(blurred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # List to store coordinates of detected zones
    coords = []

    # Browse all contours found
    for contour in contours:
        
        # Ignore small contours that could be noise
        if cv2.contourArea(contour) > NOISY_CONTOUR_LENGHT:
            
            # Get the coordinates of the rectangle enclosing the area
            x, y, w, h = cv2.boundingRect(contour)

            # Add the coordinates of the zone's vertices with a change of reference point (bottom left corner)
            coords.append([(x, height - y), (x + w, height - y), (x + w, height - y - h), (x, height - y - h)])

    return coords

In [40]:
# Execute the function to find the contours of each area
obstacles = detect_area(IMAGE_PATH, LOWER_BLACK, UPPER_BLACK)
goal_area = detect_area(IMAGE_PATH, LOWER_RED, UPPER_RED)
front_robot_area = detect_area(IMAGE_PATH, LOWER_BLUE, UPPER_BLUE)
back_robot_area = detect_area(IMAGE_PATH, LOWER_GREEN, UPPER_GREEN)

In [41]:
print(f"Coordonnées obstacles noirs :", obstacles)

print("Coordonnées zone d'arrivée :", goal_area)

print("Coordonnées zone bleue :", front_robot_area)

print("Coordonnées zone verte :", back_robot_area)

Coordonnées obstacles noirs : [[(3467, 1400), (3783, 1400), (3783, 392), (3467, 392)], [(905, 2773), (1204, 2773), (1204, 1799), (905, 1799)]]
Coordonnées zone d'arrivée : [[(1919, 2804), (3017, 2804), (3017, 1705), (1919, 1705)]]
Coordonnées zone bleue : [[(841, 1565), (1495, 1565), (1495, 241), (841, 241)]]
Coordonnées zone verte : [[(183, 1546), (857, 1546), (857, 228), (183, 228)]]
